### Import data and declare hyperparameters

In [1]:
import numpy as np

from utils import *

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


from network import *

from data_loading import *

from gaussian_predictor_levelsets import *

seed_everything(42)

In [2]:
load_path = "../data/processed_data_3Dmin/casp.npz"

X, Y = load_data(load_path)

normalize = True
splits = [0.7, 0.1, 0.1, 0.1]

subsets = split_and_preprocess(X, Y, splits=splits, normalize=normalize)

x_train, y_train, x_calibration, y_calibration, x_test, y_test, x_stop, y_stop = subsets["X_train"], subsets["Y_train"], subsets["X_calibration"], subsets["Y_calibration"], subsets["X_test"], subsets["Y_test"], subsets["X_stop"], subsets["Y_stop"]

print("X_train shape:", x_train.shape, "Y_train shape:", y_train.shape)
print("X_cal shape:", x_calibration.shape, "Y_cal shape:", y_calibration.shape)
print("X_test shape:", x_test.shape, "Y_test shape:", y_test.shape)
print("X_stop shape:", x_stop.shape, "Y_stop shape:", y_stop.shape)


d = x_train.shape[1]
k = y_train.shape[1]

n_train = x_train.shape[0]
n_test = x_test.shape[0]
n_calibration = x_calibration.shape[0]
n_stop = x_stop.shape[0]

hidden_dim = 256
hidden_dim_matrix = 256
n_hidden_layers = 3
n_hidden_layers_matrix = 1


num_epochs = 100

lr_center = 1e-3
lr_matrix = 1e-3

batch_size = 100

use_lr_scheduler = True
keep_best = True

dtype = torch.float32


x_train_tensor = torch.tensor(x_train, dtype=dtype)
y_train_tensor = torch.tensor(y_train, dtype=dtype)
x_stop_tensor = torch.tensor(x_stop, dtype=dtype)
y_stop_tensor = torch.tensor(y_stop, dtype=dtype)
x_calibration_tensor = torch.tensor(x_calibration, dtype=dtype)
y_calibration_tensor = torch.tensor(y_calibration, dtype=dtype)
x_test_tensor = torch.tensor(x_test, dtype=dtype)
y_test_tensor = torch.tensor(y_test, dtype=dtype)

alpha = 0.1

X_train shape: (32010, 7) Y_train shape: (32010, 3)
X_cal shape: (4573, 7) Y_cal shape: (4573, 3)
X_test shape: (4574, 7) Y_test shape: (4574, 3)
X_stop shape: (4573, 7) Y_stop shape: (4573, 3)


### Train the models

In [3]:
# Declare the architecture of your models and train the predictive model 

center_model = Network(d, k, hidden_dim=hidden_dim, n_hidden_layers=n_hidden_layers).to(dtype)

trainloader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_train_tensor, y_train_tensor), batch_size= batch_size, shuffle=True)
stoploader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_stop_tensor, y_stop_tensor), batch_size= batch_size, shuffle=True)
calibrationloader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_calibration_tensor, y_calibration_tensor), batch_size= batch_size, shuffle=True)
testloader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_test_tensor, y_test_tensor), batch_size= batch_size, shuffle=True)

center_model.fit_and_plot(trainloader, 
                            stoploader, 
                            num_epochs, 
                            keep_best = keep_best, 
                            lr = lr_center, 
                            verbose=True)

print("Center model trained.")

Epoch: 1, Train Loss: 0.29114311933517456, Validation Loss: 0.3018602728843689
Epoch: 2, Train Loss: 0.2846681475639343, Validation Loss: 0.29662442207336426
Epoch: 3, Train Loss: 0.26599064469337463, Validation Loss: 0.277357280254364
Epoch: 4, Train Loss: 0.25823506712913513, Validation Loss: 0.2687288820743561
Epoch: 5, Train Loss: 0.24677784740924835, Validation Loss: 0.25828099250793457
Epoch: 6, Train Loss: 0.24114175140857697, Validation Loss: 0.2523420453071594
Epoch: 7, Train Loss: 0.23863165080547333, Validation Loss: 0.2508649528026581
Epoch: 8, Train Loss: 0.22714680433273315, Validation Loss: 0.2393798828125
Epoch: 9, Train Loss: 0.21856781840324402, Validation Loss: 0.22969989478588104
Epoch: 10, Train Loss: 0.21495552361011505, Validation Loss: 0.22901980578899384
Epoch: 11, Train Loss: 0.20943695306777954, Validation Loss: 0.2228257805109024
Epoch: 12, Train Loss: 0.20912602543830872, Validation Loss: 0.2241537868976593
Epoch: 13, Train Loss: 0.2013183981180191, Validat

In [4]:
# Train the covariance matrix on top of your predictive model

matrix_model = MatrixPredictor(d, k, k, hidden_dim=hidden_dim_matrix, n_hidden_layers=0).to(dtype)

gaussian_level_sets = GaussianPredictorLevelsets(center_model, matrix_model, dtype=dtype)

gaussian_level_sets.fit(trainloader, 
                        stoploader, 
                        num_epochs = num_epochs,
                        lr_center_models = 0.0,  # This value can change if you want to update the center model
                        lr_matrix_models = lr_matrix,
                        use_lr_scheduler = use_lr_scheduler,
                        verbose = 1,
                        stop_on_best = keep_best
                        )

Epoch 0: Loss = -0.4322127103805542 - Stop Loss = 0.37005889415740967 - Best Stop Loss = 0.37005889415740967
Epoch 10: Loss = -2.139697790145874 - Stop Loss = -1.273223638534546 - Best Stop Loss = -1.341976284980774
Epoch 20: Loss = -2.1895294189453125 - Stop Loss = -1.2386797666549683 - Best Stop Loss = -1.3813153505325317
Epoch 30: Loss = -2.2191967964172363 - Stop Loss = -1.2883884906768799 - Best Stop Loss = -1.3813153505325317
Epoch 40: Loss = -2.2183196544647217 - Stop Loss = -1.3062115907669067 - Best Stop Loss = -1.3813153505325317
Epoch 50: Loss = -2.258356809616089 - Stop Loss = -1.2830098867416382 - Best Stop Loss = -1.3813153505325317
Epoch 60: Loss = -2.2880120277404785 - Stop Loss = -1.2162033319473267 - Best Stop Loss = -1.3813153505325317
Epoch 70: Loss = -2.2982425689697266 - Stop Loss = -1.2753283977508545 - Best Stop Loss = -1.3813153505325317
Epoch 80: Loss = -2.309662342071533 - Stop Loss = -1.2629238367080688 - Best Stop Loss = -1.3813153505325317
Epoch 90: Loss =

### Full output

In [5]:
gaussian_level_sets.conformalize(x_calibration=x_calibration_tensor, y_calibration=y_calibration_tensor, alpha = alpha)

tensor(2.9996)

In [6]:
coverage = gaussian_level_sets.get_coverage(x_test_tensor, y_test_tensor)
volumes  = gaussian_level_sets.get_averaged_volume(x_test_tensor).item()

print("Coverage:", coverage)
print("Average Volume:", volumes)

Coverage: 0.9147354613030171
Average Volume: 1.5655988454818726


### Revealed outputs

In [7]:
idx_knowned = np.array([0])

gaussian_level_sets.conformalize_with_knowned_idx(x_calibration=x_calibration_tensor, 
                                            y_calibration=y_calibration_tensor, 
                                            alpha = alpha, 
                                            idx_knowned=idx_knowned)

tensor(2.3888)

In [8]:
coverage = gaussian_level_sets.get_coverage_condition_on_idx(x_test_tensor, y_test_tensor)
volumes  = gaussian_level_sets.get_averaged_volume_condition_on_idx(x_test_tensor, y_test_tensor[:, idx_knowned]).item()

print("Coverage:", coverage)
print("Average Volume:", volumes)

Coverage: 0.9140795802361172
Average Volume: 1.5636283159255981


### Projection of the output

In [9]:
projection_matrix_tensor =  torch.randn((2, k), dtype=dtype)

gaussian_level_sets.conformalize_linear_projection(
                                            projection_matrix=projection_matrix_tensor,
                                            x_calibration=x_calibration_tensor, 
                                            y_calibration=y_calibration_tensor, 
                                            alpha = alpha
                                            )

tensor(2.5912)

In [10]:
coverage = gaussian_level_sets.get_coverage_projection(x_test_tensor, y_test_tensor)
volumes  = gaussian_level_sets.get_averaged_volume_projection(x_test_tensor).item()

print("Coverage:", coverage)
print("Average Volume:", volumes)

Coverage: 0.9138609532138172
Average Volume: 2.412954092025757


### Missing outputs

In [11]:
# Add NaN values to the calibration and test sets

y_calibration_nan = add_nan(y_calibration, min_nan=1, max_nan=k-1)
y_calibration_nan_tensor = torch.tensor(y_calibration_nan, dtype=dtype)

y_test_nan = add_nan(y_test, min_nan=1, max_nan=k-1)
y_test_nan_tensor = torch.tensor(y_test_nan, dtype=dtype)

In [12]:
from gaussian_predictor_missing_outputs import *

gaussian_predictor_missing_outputs = GaussianPredictorMissingValues(center_model, matrix_model, dtype=dtype)

gaussian_predictor_missing_outputs.conformalize(x_calibration=x_calibration_tensor, y_calibration=y_calibration_nan_tensor, alpha = alpha)

tensor(0.0172, dtype=torch.float64)

In [13]:
coverage_with_nan    = gaussian_predictor_missing_outputs.get_coverage(x_test_tensor, y_test_nan_tensor)
coverage_full_vector = gaussian_predictor_missing_outputs.get_coverage(x_test_tensor, y_test_tensor)

print("Coverage with NaN:", coverage_with_nan)
print("Coverage full vector:", coverage_full_vector)

Coverage with NaN: 0.9053344993441189
Coverage full vector: 0.8388718845649322
